In [1]:
from pystac_client import Client
from odc.stac import load, configure_s3_access
from dask.distributed import Client as DaskClient

from ldn.utils import (
    WGS84GRID30,
    USGSCATALOG,
    USGSLANDSAT,
    http_to_s3_url,
    mask_usgs_landsat,
    create_land_productivity_indices,
)

In [2]:
# Configure S3 access, which requires AWS credentials for loading USGS Landsat data
configure_s3_access(cloud_defaults=True, requester_pays=True)

client = Client.open(USGSCATALOG)

In [3]:
# tile = (238, 47)  # Fiji, over Suva
# tile = (79, 69)   # Martinique and St Lucia
tile = (60, 71)   # Belmopan in Belize
# tile = (239, 48)  # Southern Vanua Levu, Fiji

# Get the tile
geobox = WGS84GRID30.tile_geobox(tile)

# Zoom out (decimate) the geobox
geobox = geobox.zoom_out(10)

In [ ]:
items = client.search(
    collections=[USGSLANDSAT],
    intersects=geobox.geographic_extent,
    datetime="2022-11/2024-01",
    query={"landsat:collection_category": {"in": ["T1"]}},
).item_collection()

print(f"Found {len(items)} items")

In [ ]:
data = load(
    items,
    geobox=geobox,
    measurements=["red", "nir08", "qa_pixel"],
    chunks={"x": 2048, "y": 2048},
    groupby="solar_day",
    dtype="uint16",
    nodata=0,
    resampling={"qa_pixel": "nearest"},
    patch_url=http_to_s3_url,
)

data = data.rename_vars({"nir08": "nir"})

data

In [ ]:
# Create cloud mask, scale values to 0-1 and set nodata to NaN
masked = mask_usgs_landsat(data)

# Create the NDVI, MSAVI and EVI2
indices = create_land_productivity_indices(masked, drop=False)

indices

In [ ]:
with DaskClient(n_workers=1, threads_per_worker=16) as client:
    loaded = indices.compute()

loaded

In [ ]:
# Resample to monthly and interpolate missing values. This creates a more robust timeseries
monthly = loaded.evi2.resample(time="ME").max()
monthly = monthly.interpolate_na("time", method="linear").bfill("time").ffill("time")

monthly.plot.imshow(col="time", col_wrap=4, robust=True, cmap="viridis")

In [ ]:
# Create a spatial median
summary = monthly.median(["longitude", "latitude"])

# Plot the time series
summary.plot(ylim=(0, 1))

In [ ]:
# Create a spatial median
summary = monthly.median(["longitude", "latitude"])

# Plot the time series
summary.plot(ylim=(0, 1))

In [ ]:
# Select just the year we are interested in and integrate over time
integral_monthly = monthly.sel(time="2023").integrate("time", datetime_unit="D")

# Plot the integral
integral_monthly.plot(robust=True, cmap="Greens", size=6, vmin=0, vmax=300)

In [ ]:
from ipyleaflet import basemaps

integral_monthly.odc.explore(tiles=basemaps.Esri.WorldImagery)